In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from typing import List, Tuple

def generate_synthetic_ocr_data(
    text: str,
    width: int = 1000,
    height: int = 1400,
    font_size: int = 20,
    num_columns: int = 1,
    marginalia: bool = False,
    marginalia_probability: float = 0.2
) -> Tuple[Image.Image, List[Tuple[str, List[Tuple[int, int]]]]]:
    # Create a blank white image
    image = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(image)
    
    # Load a font
    font = ImageFont.truetype("cmr10.ttf", font_size)
    
    # Calculate column width
    column_width = width // num_columns
    
    # Split text into words
    words = text.split()
    
    # Initialize variables
    x, y = 10, 10
    line_height = font_size + 5
    current_column = 0
    text_polygons = []
    
    for word in words:
        word_width, word_height = draw.textsize(word, font=font)
        
        if x + word_width > column_width - 10:
            x = (column_width * current_column) + 10
            y += line_height
            
            if y + line_height > height - 10:
                current_column += 1
                if current_column >= num_columns:
                    break
                x = (column_width * current_column) + 10
                y = 10
        
        # Draw the word
        draw.text((x, y), word, fill='black', font=font)
        
        # Store word polygon
        text_polygons.append((word, [(x, y), (x + word_width, y),
                                     (x + word_width, y + word_height),
                                     (x, y + word_height)]))
        
        x += word_width + font_size // 2
    
    # Add marginalia if requested
    if marginalia:
        for _ in range(int(len(words) * marginalia_probability)):
            margin_x = np.random.randint(width - 200, width - 10)
            margin_y = np.random.randint(10, height - 30)
            margin_text = np.random.choice(words)
            draw.text((margin_x, margin_y), margin_text, fill='red', font=font)
            
            # Store marginalia polygon
            margin_width, margin_height = draw.textsize(margin_text, font=font)
            text_polygons.append((margin_text, [(margin_x, margin_y),
                                                (margin_x + margin_width, margin_y),
                                                (margin_x + margin_width, margin_y + margin_height),
                                                (margin_x, margin_y + margin_height)]))
    
    return image, text_polygons

def visualize_results(image: Image.Image, polygons: List[Tuple[str, List[Tuple[int, int]]]]):
    # Display original image
    plt.figure(figsize=(15, 20))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')
    
    # Display image with polygons
    plt.subplot(1, 2, 2)
    plt.imshow(image)
    for word, poly in polygons:
        x, y = zip(*poly)
        plt.plot(x + x[:1], y + y[:1], 'r-')
    plt.title("Image with Polygons")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage
text = "This is a sample text for generating synthetic OCR data. " * 20
image, polygons = generate_synthetic_ocr_data(text, num_columns=2, marginalia=True)
visualize_results(image, polygons)

OSError: cannot open resource